# Итеративные методы решения линейных систем

Ранее были расмотрены точные методы для решения линейных систем (LU разложение) и численные методы для решения систем (QR разложение, нормальная форма и SVD - разложение). Но данные методы все еще требуют достаточно емких вычислительных процессов $O(kn^3)$. Для перехода к итеративным нам потребуется понятие числа обусловленности системы.

## Число обусловленности

Число обусловленности связанно с решением системы $Ax = b$, показывает то насколько максимально хорошим будет алгоритм (допустим итеративный алгоритм) приближающий решение $x^{*}$ к истинному $x$. Число обусловленности - это, в первую очередь, свойство матрицы $A$, а не выбранного итеративного алгоритма. Для простоты, следует думать, что число обусловленности, очень грубо говоря, это скорость с которой решение  $x$ будет меняться при изменении b. Например мы хотим, чтобы, условно лучший итеративный алгоритм, cходился за сложность $O(n)$, но если наш алгоритм медленно сходится, то мы не получим никакой выгоды от использования итеративных алгоритмов относительно численных методов, рассмотренных ранее. Таким образом, если число обусловленности велико, то даже небольшая ошибка в $b$ может привести к большой ошибке в $x$.

Предположим, что у нас существует система и мы можем ее решить с какой-то точностью для коэффициента $b$, можно это сформулировать в терминах ошибки $Ax-b=e$, также можно сформулировать в терминах стабильности самого решения $A(x+\delta x) = b + A\delta x = b\pm e$

По определению, число обусловленности - это насколько сильно изменится аутпут нашего алгоритма (решение) при малом изменении инпута (коррективрока параметров модели, численная стабильность системы).

$\kappa(A) = \sup_x \frac{\frac{\|e\|}{\|b\|}}{\frac{\|\delta x\|}{\|x\|}}, e = A\delta x \rightarrow \delta x = A^{-1}e, b = Ax \rightarrow x = A^{-1}b \\
\kappa(A) = \frac{\|e\|}{\|b\|}\frac{\|A^{-1}b \|}{\|A^{-1}e\|} = \|A^{-1}\|\|A\| \geq \|A^{-1}A\| = 1$

Также может понадобиться более удобный способ нахождения числа обусловденности через сингулярные числа матрицы. Если, мы задаем отношение наших изменений через спектральную норму $L_2$, то напомню, что она формулируется следующим образом $\|A\|_2 = \sup_{x\neq0}\frac{\|Ax\|_2}{\|x\|_2}$.

Далее нам потребуется $SVD$ разложение матрицы: $A = U\Sigma V^{\top}$ с диагональными элементами $\sigma_1 \geq \sigma_2 \geq \dots \geq \sigma_n$ - сингулярные числа матрицы $A$, таким образом $\|Ax\|_2 = \| U\Sigma V^{\top}x\|_2 = \|\Sigma V^{\top}x\|_2$, так как ортогональные преобразования не влияют на норму, и введем обозначение $z =V^{\top}x \rightarrow \|Ax\|_2 = \|\Sigma z\|_2 = \sqrt{\sigma_1^2z_1^2 + \sigma_2^2z_2^2 + \dots + \sigma_n^2z_n^2}$, таким образом исходная задача трансформируется в $\sup_x \frac{\|\Sigma z\|_2}{\|z\|_2}$, что является оптимизацией взвешивания спектральных чисел, очевидно, что в максимуме весь вес вектора $z$ должен падать на первый индекс, таким образом $\|A\|_2 = \sigma_{\max}$, обратное не сложно увидеть и для обратной матрицы, что $\|A^{-1}\|_2 = \frac{1}{\sigma_{\min}}$, так как $A^{-1} = U\Sigma^{-1}V^{\top}$

$\kappa(A) = \frac{\sigma_{\max}(A)}{\sigma_{\min}(A)}$

Общая идея данного разбора состоит в том, что если мы хотим как-то быстрее посчитать систему, например итеративным алгоритмом, нам нужно сначала оценить число обусловленности для системы, если оно $\kappa(A) \approx 1$ - то система называется хорошо определенной и итеративные алгоритмы быстро сходятся, если $\kappa(A) >> 1$, то система называется плохо определенной, а итеративные алгоритмы могут плохо сходиться.

### Пример влияния числа обусловленности на сходимость степенного метода

Чтобы наглядно посмотреть и понять, насколько сильно число обусловленности влияет на сходимость итерационных алгоритмов: рассмотрим пока, что самый простой из них, алгоритм из категории **безматричных методов** (когда нам не важна структура и характеристики матрицы) - **степенной метод**. Об этом чуть позднее.

$
Ax = x \\ 
x^{(k+1)} = \frac{Ax^{(k)}}{\|Ax^{(k)}\|}
$

Зададим две матрицы: 

$
A = \begin{bmatrix}
1 & 20 \\
3 & 4
\end{bmatrix}, B = \begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}, \lambda^A_{max} \approx 10.4, \lambda^B_{max} \approx 5.4
$

In [187]:
import numpy as np
from tqdm import tqdm,trange
from time import time

def power_solve(A, l):
    
    n = A.shape[0]
    x = np.ones((n,1))
    
    for i in range(1000):
        x = np.dot(A,x)
        x_norm = np.linalg.norm(x)
        x = x/x_norm
        x_l = x.T @ A @ x
        if np.linalg.norm(l-x_l) < 1e-10:
            break
    return i,x

def return_max_eig(A):
    eigenvalues, eigenvectors = np.linalg.eig(A)
    AA = power_solve(A, np.max(np.abs(eigenvalues)))
    print(f'Iters to converge: {AA[0]}')

A = np.array([[1, 20], [3, 4]])
B = np.array([[1, 2], [3, 4]])
return_max_eig(A), return_max_eig(B);

Iters to converge: 37
Iters to converge: 8


## Итерационные методы. Ax = b

До сих пор наш подход к решению $Ax = b$ был прямым. Мы работали с матрицей A путем вычитания строк или вывода через нормальную форму. Итерационные методы заменяют матрицу A более простой матрицей S. Разность $T = S - A$ переносится в правую часть уравнения. Задача становится проще для решения, когда вместо A используется S. Но за это приходится платить - более простую систему приходится решать снова и снова.

$
Ax = b \rightarrow Sx = b - Tx \\
Sx^{(k+1)} = b - Tx^{(k)}
$

Мы начинаем с приближения $x_0$, решаем систему $Sx^{(1)} = Tx^{(0)}+b$ относительно $x^{(1)}$ продолжаем системой $Sx^{(2)} = b - Tx^{(2)}$ и так далее до сходимости последовательности $x^{(0)}, x^{(1)}, \dots, x^{(k)}$. 

Здесь возникают две основные задачи, как построить $A = S + T$ - скорость вполнения шага и скорость сходимости. Скорость каждого шага зависит от матрицы $S$, так как именно ее мы используем как оснонвую для решения системы, а скорость сходимости зависит от $S^{-1}T$. Матрица $S$ для такого вида задач называет **предобуславливающей матрицей** (preconditioner) системы $A$ - и она должна быть диагональной либо нижнетреугольной. А ошибка $e^{(k)} = x - x^{(k)}$ должна быстро сходиться к нулю.

$
Se^{(k+1)} = Te^{(k)} - \text{Уравнение ошибки}
$

Таким образом на каждом шаге на ошибку влияет преобразование $S^{-1}T$, если оно мало - то ошибка быстро сходится к нулю. Например, если рассмотреть пример где $T = 0$, тогда $S = A$ и первый шаг итерации это решение системы $Ax=b$ и после этого шага ошибка сразу же будет нулевой. Но стоимость этого шага будет довольно высока, чего мы и хотим избежать. Поэтому выбор $S$ это борьба между скорость каждого шага (простая конфигурация матрицы $S$) и быстрой сходимости ошибки (матрица $A$ была похожа на $S$). Существует три простых способа задать матрицу $S$:

1. $S$ - диагональные элементы матрицы $A$ (такие итерации называются **методом Якоби**)
2. $S$ - нижнетреугольная часть матрицы $A$ (это называется  **методом Гаусса-Зейделя**
3. $S$ - является неполным $LU$ - разложением матрицы A. 

Уравнение $e^{(k+1)} = S^{-1}Te^{(k)}$: на каждой итерации умножается на одну и ту же матрицу каждый шаг $B = S^{-1}T$, и тогда форму ошибки можно переписать к следующему виду $e^{(k)} = B^{k}e^{(0)}$, тогда если $B^{k} \rightarrow 0, e^{(k)} \rightarrow 0$, такое достижимо когда $\forall \lambda_B : |\lambda_B| < 1$, а уровнем сходимости называется **спектральный радиус** $\rho = \max |\lambda_B|$.

Доказать это несложно, при заданной итеративной системе $x^{(k)} = Bx^{(k-1)}+b  = B(Bx^{(k-2)}+b)+b = \dots = B^{k}x^{(0)} + (B^{k-1} + \dots + B^1 + I)c = (I-T)^{-1}c$


### Метод Якоби

Как и ранее было сказано - метод Якоби определяет предобславливающую матрицу, как диагональ матрицы $A$.

Решаем следующую систему:

$
\begin{cases}
a_{11}x_1 + a_{12}x_2 + \dots + a_{1n}x_n = b_1 \\
a_{21}x_1 + a_{22}x_2 + \dots + a_{2n}x_n = b_2 \\
\vdots \\
a_{n1}x_1 + a_{n2}x_2 + \dots + a_{nn}x_n = b_n
\end{cases}
$

Просто через выражение переменных:

$
\begin{cases}
x_1 = \frac{1}{a_{11}}(b_1 - a_{12}x_2  \dots - a_{1n}x_n) \\
x_2 = \frac{1}{a_{22}}(b_2 - a_{21}x_1 - \dots - a_{2n}x_n) \\
\vdots \\
x_n = \frac{1}{a_{nn}}(b_n - a_{n1}x_1 - \dots - a_{nn-1}x_{n-1})
\end{cases}
$

Указывая изначальную инициализацию $x^{(0)} = (x_1^{(0)}, x_2^{(0)}, \dots, x_n^{(0)})$ итерируемся по системе получая $x^{(k)} = (x_1^{(k)}, x_2^{(k)}, \dots, x_n^{(k)})$, либо в следующей форме:

$
x_i^{(k)} = \frac{1}{a_{ii}}\left[-\sum_{j=1}^n(a_{ij}x_j^{(k-1)}) + b_i\right]$

Либо в матричной форме:

$
S = Diag(A), \quad T = A - Diag(A) \\
x^{(k+1)} = S^{-1}(b - Tx^{(k)})
$



In [239]:
def jacobi_solve(A, b, k, early_stop = False, print_time = False):
    
    n = A.shape[0]
    x = np.ones((n,1))
    x_prev = np.random.rand(n,1)
    S = np.zeros_like(A) + np.diag(np.diag(A))
    T = A - S
    S_inv = np.linalg.inv(S)
    
    max_abs_eigen = np.abs(np.linalg.eig(np.dot(S_inv, T))[0]).max()
    
    if max_abs_eigen > 1:
        print(max_abs_eigen)
        
        raise AttributeError('Max eigen > 1')
        
    start = time()
    
    for i in range(k):
        Tx = np.dot(T,x)
        Txb = b - Tx
        x = np.dot(S_inv, Txb)
        
        if early_stop:
            if np.linalg.norm(x-x_prev) < 1e-5:
                break
        x_prev = x
        
    if print_time:
        print(f'{1000*time() - 1000*start} ms')
        
    return x

In [240]:
n = 20
A = np.ones((n,n))
diag_ = np.diag([float(i*j) for i, j in zip(27*np.ones((n,1)),np.ones((n,1)))])
A = A + diag_
b = np.random.randint(n, size = (n,1))

In [241]:
solution_ = jacobi_solve(A,b, k = 2000, early_stop = True, print_time = True)

0.68212890625 ms


In [242]:
%%time
linalg_solution = np.linalg.solve(A,b)

CPU times: user 469 ms, sys: 46.8 ms, total: 515 ms
Wall time: 165 ms


In [246]:
print(f'Error from linalg solution: {(np.linalg.norm(solution_ - linalg_solution)):.6f}')

Error from linalg solution: 0.000003


### Метод Гаусса-Зейделя

В методе **Якоби** значения $x_i^{(k)}$ полученные на шаге $k$ не использовались до наступления шага $(k+1)$. В методе **Гаусса-Зейделя** предлагается использовать $x_i^{(k)}$ на текущем шаге $k$ для следующих полсе $i$ переменных, например использовать обновленный $x_1^{(k)}$ для расчета $x_2^{(k)}$.

$
\begin{cases}
a_{11}x_1^{(k)} = b_1 - a_{12}x_2^{(k-1)} - \dots - a_{1n}x_n^{(k-1)} \\
a_{21}x_1^{(k)} + a_{22}x_2^{(k)} = b_2 - a_{23}x_3^{(k-1)} - \dots - a_{2n}x_n^{(k-1)} \\
\vdots \\
a_{n1}x_1^{(k)} + a_{n2}x_2^{(k)} + \dots a_{nn-1}x_{n-1}^{(k)} = b_n - a_{nn}x_{n}^{(k)}
\end{cases}
$

Процесс обновления можно записать в следующем виде:

$
x_i^{(k)} = \frac{1}{a_{ii}}\left[-\sum_{j=1}^{i-1}(a_{ij}x_j^{(k)}) - \sum_{j=i+1}^{n}(a_{ij}x_j^{(k-1)})+ b_i\right]
$

В данном методе мы представляем матрицу $A = D + L + U$, как сумму диагональной нижне и верхне треугольных матриц:

$
(D + L + U)x = b \\
(D + L)x = b - Ux \\
x^{(k+1)} = (D + L)^{-1}(b - Ux^{(k)})
$

In [252]:
def solve_gs(A, b, k, early_stop = False, print_time = False):

    n = A.shape[0]
    x = np.ones((n,1))
    x_prev = np.random.rand(n,1)
    
    D = S = np.zeros_like(A) + np.diag(np.diag(A))
    DL = np.tril(A, -1) + D
    U = A - DL
    DL_inv = np.linalg.inv(DL)
    
    max_abs_eigen = np.abs(np.linalg.eig(np.dot(DL_inv, U))[0]).max()
    
    if max_abs_eigen > 1:
        print(max_abs_eigen)
        
        raise AttributeError('Max eigen > 1')
        
    start = time()
    for i in range(k):
        for ix in range(n):
            sum_ = 0.
            for j in range(n):            
                if ix != j:
                    sum_ += A[ix,j]*x[j]
            x[ix] = (b[ix]-sum_)/A[ix,ix]
        
        if early_stop:
            
            if np.linalg.norm(x-x_prev) < 1e-5:
                    break
        x_prev = x.copy()
    
    if print_time:
        print(f'{1000*time() - 1000*start} ms')
         
    return x

In [253]:
solution_gs = solve_gs(A,b,k = 2000, early_stop = True, print_time = True)

15.22998046875 ms


In [254]:
print(f'Error from linalg solution: {(np.linalg.norm(solution_gs - linalg_solution)):.6f}')

Error from linalg solution: 0.000002


In [255]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [256]:
A = np.array([[4., -3], [2, 5]])
b = np.array([[-1., 19]])
true_sol = np.linalg.solve(A, b.T)
fig, ax = plt.subplots(figsize=(12,5))
def animate(iter_):
    ax.cla()
    ax.set_xlim(-4, 4)
    ax.set_ylim(-5, 5)
    x1 = jacobi_solve(A, b.T, iter_)
    x2 = solve_gs(A, b.T, iter_)
    ax.plot(x1[0][0], x1[1][0], 'o', color='b', label='Jacobi', ms=8)
    ax.plot(x2[0][0], x2[1][0], 'x', color='r', label='Gauss-Seidel', ms=8)
    ax.plot(true_sol[0][0], true_sol[1][0], 's', color='green', ms=8, label='True solution')
    ax.axhline()
    ax.axvline()
    ax.set_title('Jacobi and Gauss Seidel methods: iteration step '+str(iter_), size=16)
    ax.set_xlabel('$x_1$', size=16)
    ax.set_ylabel('$x_2$', size=16)
    ax.legend(loc='upper right')
plt.close()
anim = FuncAnimation(fig, animate, frames=30, interval=500)
HTML(anim.to_jshtml())